In [ ]:
# Importing Libraries

# Data processing

import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Visualization

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Machine learning

from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn.feature_selection as fs
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Changing Not vailable to NaN

df = df.replace('Not Available', None)

# Removing the character in the data.

df[['DOF Gross Floor Area (ft²)', 'Year Built']] = df[['DOF Gross Floor Area (ft²)', 'Year Built']].replace(',','', regex=True)

# Selecting Obj type which are actually numbers

cols = ['NYC Building Identification Number (BIN)','Postal Code','DOF Gross Floor Area (ft²)','Year Built','ENERGY STAR Score','Source EUI (kBtu/ft²)','Weather Normalized Site EUI (kBtu/ft²)','Site EUI (kBtu/ft²)','Weather Normalized Source EUI (kBtu/ft²)','Weather Normalized Site Electricity Intensity (kWh/ft²)','Weather Normalized Site Natural Gas Intensity (therms/ft²)','Natural Gas Use (kBtu)','Weather Normalized Site Natural Gas Use (therms)','Electricity Use - Grid Purchase (kWh)','Weather Normalized Site Electricity (kWh)','Total GHG Emissions (Metric Tons CO2e)','Direct GHG Emissions (Metric Tons CO2e)','Indirect GHG Emissions (Metric Tons CO2e)','Water Use (All Water Sources) (kgal)','Water Use Intensity (All Water Sources) (gal/ft²)']


# Selecting Obj type which are actually numbers

cols2 = ['BBL - 10 digits','Street Number', 'Self-Reported Gross Floor Area (ft²)','Largest Property Use Type - Gross Floor Area (ft²)','2nd Largest Property Use - Gross Floor Area (ft²)','3rd Largest Property Use Type - Gross Floor Area (ft²)','Fuel Oil #1 Use (kBtu)','Fuel Oil #2 Use (kBtu)','Fuel Oil #5 & 6 Use (kBtu)','Fuel Oil #2 Use (kBtu)','Diesel #2 Use (kBtu)','Propane Use (kBtu)','District Steam Use (kBtu)','District Chilled Water Use (kBtu)','Electricity Use - Grid Purchase (kBtu)','Annual Maximum Demand (kW)']

# Converting Obj type to Num

col = cols + cols2
df[col] = df[col].apply(pd.to_numeric, errors='coerce', axis=1)


Sum = df.isnull().sum()
Percentage = ( df.isnull().sum()/df.isnull().count())

nulls = pd.concat([Sum,Percentage], axis =1, keys= ['Sum', 'Percentage']).sort_values(by='Sum', ascending=False).head(30)
nulls

#Selecting the columns with more than 30% missing values

remove = nulls[nulls['Percentage']>=0.30]
remove.index.name = 'Features' # Renames the index to 'Features'
remove.reset_index(level=0, inplace=True) # Re-index's the dataframe, turning 'Features' into a column
remove

remove['Features'].tolist()


#Dropping the columns with more than 30% missing values

df = df.drop(remove['Features'].tolist(), axis=1)


# Dropping other columns that aren't useful

df = df.drop(['Order','Property Name', 'Parent Property Id','Parent Property Name', 'Property Id','BBL - 10 digits','NYC Building Identification Number (BIN)','Postal Code','Street Number', 'NYC Borough, Block and Lot (BBL) self-reported','Address 1 (self-reported)','Water Required', 'Address 2 (self reported)','Street Name','Fuel Oil #4 Use (kBtu)','Kerosene Use (kBtu)', 'Annual Maximum Demand (MM/YYYY)', 'Generation Date','District Hot Water Use (kBtu)'], axis=1)


df_obj = df.select_dtypes(include=['object'])
df_num = df.select_dtypes(exclude=['object'])

# Creating a pipeline that handles NaNs as well as transforms data using standard scaler.

num_pipeline = Pipeline([('imputer', SimpleImputer(strategy="median")), ('std_scaler', StandardScaler())])

df_pipeline = num_pipeline.fit_transform(df_num)

df_pipe = pd.DataFrame(df_pipeline, columns=df_num.columns,index=df_num.index)

le=LabelEncoder()

df_cleaned = pd.concat([df_pipe,df_obj], axis =1)
df_cleaned.dropna(inplace=True)
df_obj = df_cleaned.select_dtypes(include=['object'])

for col in df_obj.columns.values:
    # Encoding only categorical variables
    if df_obj[col].dtypes=='object':
        # Using whole data to form an exhaustive list of levels
        data=df_obj[col]
        le.fit(data.values)
        df_obj[col]=le.transform(df_obj[col])
        
df_cleaned = pd.concat([df_pipe,df_obj], axis =1)

df_cleaned.dropna(inplace=True)

#Selecting features and the target variable

y = df_cleaned['Total GHG Emissions (Metric Tons CO2e)']  #Target Feature
X = df_cleaned.drop("Total GHG Emissions (Metric Tons CO2e)", 1)

selected_df = X[['DOF Gross Floor Area (ft²)',
 '2nd Largest Property Use - Gross Floor Area (ft²)',
 '3rd Largest Property Use Type - Gross Floor Area (ft²)',
 'ENERGY STAR Score', 
 'Weather Normalized Source EUI (kBtu/ft²)',
 'Fuel Oil #2 Use (kBtu)',
 'Fuel Oil #5 & 6 Use (kBtu)',
 'District Steam Use (kBtu)',
 'District Chilled Water Use (kBtu)',
 'Weather Normalized Site Natural Gas Use (therms)',
 'Electricity Use - Grid Purchase (kWh)',
 'Annual Maximum Demand (kW)',
 'Indirect GHG Emissions (Metric Tons CO2e)',
 'Water Use (All Water Sources) (kgal)',
 'Primary Property Type - Self Selected']]

data = pd.concat([y, selected_df], axis= 1)
data